<a href="https://colab.research.google.com/github/recanoy/Coursera_AlgorithmsForDNASequencing/blob/main/Week2_LectureNotes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Author: Raymart Jay E. Canoy
# Date: 16 October 2022
___

* **Outline**
  1. **Boyer-Moore Algorithm**: An alternative to naive algorithm
  2. **Indexing**: Working on genomes and read-alignment problem
  3. **Pigeon-hole Principle**: On exact and approximare matching problems


### (1) Boyer-Moore basics

* Learn from character comparisons to skip pointless alignments
* Try alignments in left-to-right order, and try character comparisons in right-to-left order
* **Bad character rule:**
  Upon mismatch, skip alignments until
  (a) mismatch becomes a match, or
  (b) P moves past mismatched character
* **Good suffix rule:**
  Let $t=$ substring matched by inner loop; skip until
  (a) there are no mismatches between $P$ and $t$ or
  (b) $P$ moves past $t$

#### Diversion: Repetitive elements

* Transposable elements: Infiltrates the genome
* 45% in the human genome came from transposable elements
  * For example, `Alu` (11%)
  * Read: Cordaux R, Batzer MA. The impact of retrotransposons on human genome evolution. Nat Rev Genet. 2009 Oct; 10(10):691-703
* **Relevance of transposable elements**
  * Transposable elements create ambiguity in the human genome
  * Repettive sequence in the genome are gonna create problem in our problem.

In [13]:
import string

def z_array(s):
  """
  Use Z algorithm (Gusfield theorem 1.4.1) to preprocess s
  """
  z = [len(s)] + [0] * (len(s)-1)

  # Initial comparison of s[1:] with prefix
  for i in range(1, len(s)):
    if s[i] == s[i-1]:              # compares the nucleotide in the previos index matches with the current nucleotide
      z[1] += 1                     # if yes, add 1 to the the z list at index 1
    else:                           # otherwise, break
      break

  r, l = 0, 0
  if z[1] > 0:
    r, l = z[1], 1
  
  for k in range(2, len(s)):
    assert z[k] == 0
    if k > r:
      # Case 1
      for i in range(k, len(s)):
        if s[i] == s[i-k]:
          z[k] += 1
        else:
          break
      r, l = k + z[k] - 1, k
    else:
      # Case 2
      # Calculate length of beta
      nbeta = r - k + 1
      zkp = z[k-l]
      if nbeta > zkp:
        # Case 2a: Zkp wins
        z[k] = zkp
      else:
        # Case 2b: Compare characters just past r
        nmatch = 0
        for i in range(r+l, len(s)):
          if s[i] == s[i-k]:
            nmatach += 1
          else:
            break
        l, r = k, r + nmatch
        z[k] = r - k + 1
  
  return z


def n_array(s):
  """
  Compare the N array (Gusfield theorem 2.2.2) from the Z array
  """
  return z_array(s[::-1])[::-1]

def big_l_prime_array(p, n):
  """
  Compile L' array (Gusfield theorm 2.2.2) using p and N array.
  L'[i] = largest index j less than n such that N[j] = |P[i:]|
  """
  lp = [0] * len(p)
  for j in range(len(p)-1):
    i = len(p) - n[j]
    if i < len(p):
      lp[i] = j + 1

  return lp

def big_l_array(p, lp):
  """
  Compile L array (Gusfield theorem 2.2.2) using p and L' array.
  L[i] = largest index j less than n such that N[j] >= |P[i:]|
  """
  l = [0] * len(p)
  l[1] = lp[1]
  for i in range(2, len(p)):
    l[i] = max([i-1], lp[i])
  
  return l

def small_l_prime_array(n):
  """
  Compile lp' array (Gusfield theorem 2.2.4) using N array.
  """
  small_lp = [0] * len(n)
  for i in range(len(n)):
    if n[i] == i+1:                 # prefix matching a suffix
      small_lp[len(n)-i-1] = i+1
  for i in range(len(n)-2, -1, -1): # "smear" them out to the left
    if small_lp[i] == 0:
      small_lp[i] = small_lp[i+1]
  
  return small_lp

def good_suffix_table(p):
  """
  Return tables needed to apply good suffix rule.
  """
  n = n_array(p)
  lp = big_l_prime_array(p, n)
  return lp, big_l_array(p, lp), small_l_prime_array(n)

def good_suffix_mismatch(i, big_l_prime, small_l_prime):
  """
  Given a mismatch at offset i, and given L/L' and l' arrays,
  return amount to shift as determined by good suffix rule.
  """
  length = len(big_l_prime)
  assert i < length
  if i == length - 1:
    return 0
  i += 1      # i points to leftmost matchig position of p
  if big_l_prime[i] > 0:
    return length - big_l_prime[i]
  return length - small_l_prime[i]

def good_suffix_match(small_l_prime):
  """
  Given a full match of P to T, return amount to shift as
  determined by good suffix rule.
  """
  return len(small_l_prime) - small_l_prime[1]

def dense_bad_char_tab(p, amap):
  """
  Given pattern string and list with ordered alphabet characters, create
  and return a dense bad character table. Table is indexed by offset
  then by character.
  """
  tab = []
  nxt = [0] * len(amap)
  for i in range(0, len(p)):
    c = p[i]
    assert c in amap
    tab.append(nxt[:])
    nxt[amap[c]] = i + 1
  return tab